# 메타클래스로 클래스의 존재를 등록하자

메타클래스를 사용하는 또 다른 일반적인 사례는 프로그램에 있는 타입을 자동으로 등록하는 것이다. 등록(registration)은 간단한 식별자(identifier)를 대응하는 클래스에 매핑하는 역방향 조회(reverse lookup)를 수행할 때 유용하다.

예를 들어 Python 객체를 직렬화한 표현을 JSON으로 구현한다고 해보자.

In [4]:
import json

class Serializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({'args': self.args})
    

간단한 불변 자료 구조를 문자열로 쉽게 직렬화할 수 있다.

In [5]:
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Point2D(%d, %d)' % (self.x, self.y)
    
point = Point2D(5, 3)
print('Object:    ', point)
print('Serialized:', point.serialize())

Object:     Point2D(5, 3)
Serialized: {"args": [5, 3]}


역직렬화하는 또 다른 클래스

In [6]:
class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])

Deserializable을 이용하면 간단한 불변 객체들을 범용적인 방식으로 쉽게 직렬화하고 역직렬화 할 수 있다.

In [8]:
class BetterPoint2D(Deserializable):
    # ...
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Point2D(%d, %d)' % (self.x, self.y)
    
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])
    

point = BetterPoint2D(5, 3)
print('Before:     ', point)
data = point.serialize()
print('Serialized: ', data)
after = BetterPoint2D.deserialize(data)
print('After:      ', after)

Before:      Point2D(5, 3)
Serialized:  {"args": [5, 3]}
After:       Point2D(5, 3)


직렬화된 데이터에 대응하는 타입을 미리 알고 있을 경우에만 대응 가능하다는 문제가 있다. 이상적으로 JSON으로 직렬화되는 클래스를 많이 갖추고 그중 어떤 클래스든 대응하는 Python 객체로 역직렬화하는 공통 함수를 하나만 두려고 할 것이다.

이렇게 만들려면 직렬화할 객체의 클래스 이름을 JSON 데이터에 포함시키면 된다.

In [13]:
class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })
    
    def __repr__(self):
        return 'Point2D(%d, %d)' % (self.x, self.y)

다음으로 클래스 이름을 해당 클ㄹ래스의 객체 생성자에 매핑하고 이 매핑을 관리한다. 범용 역직렬화 함수는 register_class에 넘긴 클래스가 어떤 것이든 제대로 동작한다.

In [14]:
registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

deserialize가 항상 제대로 동작함을 보장하려면 추후에 역직렬화할 법한 모든 클래스에 register_class를 호출해야 한다.

In [15]:
class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
register_class(EvenBetterPoint2D)

이제 어떤 클래스를 담고 있는지 몰라도 임의의 JSON 문자열을 역직렬화할 수 있다.

In [16]:
point = EvenBetterPoint2D(5, 3)
print('Before:     ', point)
data = point.serialize()
print('Serialized: ', data)
after = deserialize(data)
print('After:      ', after)

Before:      Point2D(5, 3)
Serialized:  {"class": "EvenBetterPoint2D", "args": [5, 3]}
After:       Point2D(5, 3)


In [17]:
class Point3D(BetterSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

In [18]:
point = Point3D(5, 9, -4)
data = point.serialize()
deserialize(data)

KeyError: 'Point3D'

BetterSerializable를 상속해서 서브클래스를 만들더라도 class 문의 본문 이후에 register_class를 호출하지 않으면 실제로 모든 기능을 사용하진 못한다.

프로그래머가 의도한 대로 BetterSerializable을 사용하고 모든 경우에 `register_class`가 호출된다고 확신 하기 위해 메타클래스를 이용하면 서브클래스가 정의될 때 class 문을 가로채는 방법으로 만들 수 있다. 메타 클래스로 클래스 본문이 끝나자마자 새 타입을 등록하면 된다.

In [19]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls
    
class RegisteredSerializable(BetterSerializable,
                            metaclass=Meta):
    pass

RegisterSerializable의 서브클래스를 정의할 때 register_class가 호출되어 deserialize가 항상 기대한 대로 동작할 것이라고 확신할 수 있다.

In [21]:
class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z
        
v3 = Vector3D(10, -7, 3)
print('Before:     ', v3)
data = v3.serialize()
print('Serialized: ', data)
after = deserialize(data)
print('After:      ', deserialize(data))

Before:      Point2D(10, -7)
Serialized:  {"class": "Vector3D", "args": [10, -7, 3]}
After:       Point2D(10, -7)


메타클래스를 이용해 클래스를 등록하면 상속 트리가 올바르게 구축되어있는 한 클래스 등록을 놓치지 않는다. 앞에서 본 것처럼 직렬화에 잘 동작하며 데이터베이스 객체 관계 매핑(`ORM`, `Object-Relationship Mapping`), 플러그인 시스템, 시스템 후크에도 적용할 수 있다.